In [4]:
import snowflake.connector
import pandas as pd

# 연결 설정
conn = snowflake.connector.connect(
    user='namgonlee_svc',
    account='MAESGVY-TY96489',
    private_key_file='/Users/encr25n10048/Documents/namgonlee_svc_key.p8'
    # warehouse='USER_XSMALL',
    # database='YOUR_DATABASE',
    # schema='YOUR_SCHEMA'
)

In [15]:
# sql 문 안에 변수처리하면 작동하지 않기 때문에 아래와 python 에서 변수처리 한 뒤 sql에는 파이썬 변수를 넣어야 한다.
# 변수list
# 변수 사용시 -> '{변수명}'
abtestId = 'ab_6821560d42161e1e5fb42456'           
start_date = '2025-05-13'                          
end_date = '2025-06-23'                            

# SQL 실행 및 결과 불러오기
query = f'''

WITH 

log_base AS (

    SELECT
        BASE_DATE,
        EVENTTIME,
        PCID,
        ABTEST,
        TESTTYPE,
        SCREENNAME,
        EVENTTYPE,
        EVENTNAME,
        EVENTNAMEGROUP,
        STATUS,
        OS_TYPE,
        OS_DETAIL,
        NULLIF(TRIM(f.value::STRING), '') AS ABTEST_SPLIT,
        SPLIT_PART(ABTEST_SPLIT, '_', 1) || '_' || SPLIT_PART(ABTEST_SPLIT, '_', 2) AS ABTEST_ID,
        SPLIT_PART(ABTEST_SPLIT, '_', 3) AS ABTEST_GROUP,
    FROM
        ENCAR.ANALYSIS_MART.ESTIMATE_SESSION_STAGE AS ess,
        LATERAL FLATTEN(input => SPLIT(ess.ABTEST, ',')) AS f
    WHERE
        1=1
        AND BASE_DATE BETWEEN '{start_date}' AND '{end_date}'
        AND NOT (ess.OS_TYPE = 'app' AND ess.OS_DETAIL = 'ios')         -- App-ios 는 전부 Group A에만 분배된 이슈 발견되었기 때문에 이는 제외하려고 함
)

SELECT
    base_date,
    COUNT(DISTINCT PCID) AS CNT_PCID
FROM
    log_base
GROUP BY
    ALL
ORDER BY
    1

'''
df = pd.read_sql(query, conn)

# 결과 확인
df

/var/folders/n9/bgd6x1s52gv1g9c2y_y9l4dr0000gn/T/ipykernel_43646/4152899274.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,BASE_DATE,CNT_PCID
0,2025-05-13,19754
1,2025-05-14,19009
2,2025-05-15,18077
3,2025-05-16,17396
4,2025-05-17,15838
5,2025-05-18,16480
6,2025-05-19,19483
7,2025-05-20,18785
8,2025-05-21,18565
9,2025-05-22,18107
